In [1]:
from os import path
from os.path import join
from keras.models import load_model
import layers_builder as layers
import numpy as np
import argparse
import os


Using TensorFlow backend.
/home/baskar/Documents/DL/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/baskar/Documents/DL/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/baskar/Documents/DL/venv/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/baskar/Documents/DL/venv/l

In [2]:
import os
import random
import numpy as np
from scipy.misc import imresize, imread
from scipy.ndimage import zoom
from collections import defaultdict

In [3]:
def update_inputs(batch_size = None, input_size = None, num_classes = None):
  return np.zeros([batch_size, input_size[0], input_size[1], 3]), \
    np.zeros([batch_size, input_size[0], input_size[1], num_classes])

In [4]:
def data_generator(datadir='', nb_classes = None, batch_size = None, input_size=None, separator='_', test_nmb=50):
  if not os.path.exists(datadir):
    print("ERROR!The folder is not exist")

  # Path of the train_val text file
  train_txt_filename = 'dataset/ImageSets/Segmentation/trainval.txt'

  # Taking the list of all images from the text file
  with open(train_txt_filename, 'r') as f:
      lines = f.readlines()
  img_name_list = []
  for line in lines:
      img_name_list.append(line.strip())

  # Forming the full file paths for all the trainval images
  full_path = 'dataset/JPEGImages'
  extention = '.jpg'
  # image_paths = list(map(lambda x: os.path.join(full_path, x) + extention, img_name_list))

  anno_path = 'dataset/SegmentationClass'
  ext = '.png'
  # anno_paths = list(map(lambda x: os.path.join(anno_path, x) + ext, img_name_list))

  data_dict = {}
  for img_name in img_name_list:
    data_dict[img_name] = [os.path.join(full_path, img_name) + extention,
                          os.path.join(anno_path, img_name) + ext]

  print(len(data_dict))

  # for image_path in image_paths:
  #   # nmb = image_path.split(separator)[0]
  #   nmb = image_path.split('.jpg')[0].split('/')[-1]
  #   data[nmb]['image'] = image_path
  
  # anno_dir = os.path.join(datadir, "maps_bordered")
  # anno_paths = os.listdir(anno_dir)
  # for anno_path in anno_paths:
  #   nmb = anno_path.split(separator)[0]
  #   data[nmb]['anno'] = anno_path
  # values = data.values()
  values = data_dict.values()
  
  print(len(values))
  # random.shuffle(values)
  return generate(list(values)[:50], nb_classes, batch_size, input_size)

In [5]:
def generate(values, nb_classes, batch_size, input_size):
  while 1:
    # random.shuffle(values)
    images, labels = update_inputs(batch_size=batch_size,
       input_size=input_size, num_classes=nb_classes)
    for i, pair in enumerate(values):
      img = imresize(imread(pair[0], mode='RGB'), input_size)
      y = imread(pair[1], mode='L')
      # y = imresize(imread(pair[1]), mode='L'), input_size)
      h, w = input_size
      y = zoom(y, (1.*h/y.shape[0], 1.*w/y.shape[1]), order=1, prefilter=False)
      y = (np.arange(nb_classes) == y[:,:,None]).astype('float32')
      assert y.shape[2] == nb_classes
      images[i % batch_size] = img
      labels[i % batch_size] = y
      if (i + 1) % batch_size == 0:
        yield images, labels
        images, labels = update_inputs(batch_size=batch_size,
          input_size=input_size, num_classes=nb_classes)


In [6]:
def train(datadir, logdir, input_size, nb_classes, resnet_layers, batchsize, sep):
    
    model = layers.build_pspnet(nb_classes=nb_classes,
                                resnet_layers=resnet_layers,
                                input_shape=input_size)
    # dataset_len = len(os.listdir(os.path.join(datadir, 'imgs')))
    model.summary()
    train_generator = data_generator(
        datadir=datadir, batch_size=batchsize, input_size=input_size, nb_classes=nb_classes, separator=sep)
    model.fit_generator(
        generator=train_generator,
        epochs=2, verbose=True, steps_per_epoch=1)
    model.save_weights('pspnet.h5')
    print('Model saved!!!')


In [7]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
datadir = 'dataset'
logdir = 'logs'
input_size = (473,473)
classes = 21
nb_resnet_layers = 101
batch = 1
model = None
sep = '.'
train(datadir, logdir, input_size, classes, nb_resnet_layers,
      batch, sep)

Building a PSPNet based on ResNet 101 expecting inputs of shape (473, 473) predicting 21 classes




PSP module will interpolate to a final feature map size of (60, 60)

tttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
new size:
(60, 60)

tttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
new size:
(60, 60)
tttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
new size:
(60, 60)
tttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
new size:
(60, 60)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
tttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
new size:
[473, 473]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


__________________________________________________________________________________________________
Layer (type)                    Output Shape 

conv4_4_1x1_increase_bn (BatchN (None, 60, 60, 1024) 4096        conv4_4_1x1_increase[0][0]       
__________________________________________________________________________________________________
add_11 (Add)                    (None, 60, 60, 1024) 0           conv4_4_1x1_increase_bn[0][0]    
                                                                 activation_34[0][0]              
__________________________________________________________________________________________________
activation_37 (Activation)      (None, 60, 60, 1024) 0           add_11[0][0]                     
__________________________________________________________________________________________________
conv4_5_1x1_reduce (Conv2D)     (None, 60, 60, 256)  262144      activation_37[0][0]              
__________________________________________________________________________________________________
conv4_5_1x1_reduce_bn (BatchNor (None, 60, 60, 256)  1024        conv4_5_1x1_reduce[0][0]         
__________

__________________________________________________________________________________________________
activation_64 (Activation)      (None, 60, 60, 1024) 0           add_20[0][0]                     
__________________________________________________________________________________________________
conv4_14_1x1_reduce (Conv2D)    (None, 60, 60, 256)  262144      activation_64[0][0]              
__________________________________________________________________________________________________
conv4_14_1x1_reduce_bn (BatchNo (None, 60, 60, 256)  1024        conv4_14_1x1_reduce[0][0]        
__________________________________________________________________________________________________
activation_65 (Activation)      (None, 60, 60, 256)  0           conv4_14_1x1_reduce_bn[0][0]     
__________________________________________________________________________________________________
zero_padding2d_21 (ZeroPadding2 (None, 64, 64, 256)  0           activation_65[0][0]              
__________

activation_89 (Activation)      (None, 60, 60, 256)  0           conv4_22_1x1_reduce_bn[0][0]     
__________________________________________________________________________________________________
zero_padding2d_29 (ZeroPadding2 (None, 64, 64, 256)  0           activation_89[0][0]              
__________________________________________________________________________________________________
conv4_22_3x3 (Conv2D)           (None, 60, 60, 256)  589824      zero_padding2d_29[0][0]          
__________________________________________________________________________________________________
conv4_22_3x3_bn (BatchNormaliza (None, 60, 60, 256)  1024        conv4_22_3x3[0][0]               
__________________________________________________________________________________________________
activation_90 (Activation)      (None, 60, 60, 256)  0           conv4_22_3x3_bn[0][0]            
__________________________________________________________________________________________________
conv4_22_1

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/2


/home/baskar/Documents/DL/venv/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  import sys
/home/baskar/Documents/DL/venv/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  import sys
/home/baskar/Documents/DL/venv/lib/python3.5/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


1/1 [==============================] - 29s 29s/step - loss: 0.4646 - acc: 0.0081
Epoch 2/2
1/1 [==============================] - 12s 12s/step - loss: 3.4103 - acc: 0.0074
